In [1]:
import tensorflow as tf
print('version tensorflow : ',tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import save_model
import glob
import os 
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline

2022-12-04 10:34:53.817425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 10:34:54.538244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 10:34:54.538290: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-04 10:34:56.406489: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

version tensorflow :  2.11.0


In [2]:
#data sheet
base_dir = os.path.join('DATASET/')
downdog_dir = os.path.join(base_dir,'downdog/')
goddes_dir = os.path.join(base_dir,'goddess/')
plank_dir = os.path.join(base_dir,'plank/')
tree_dir = os.path.join(base_dir,'tree/')
warrior2_dir = os.path.join(base_dir,'warrior2/')

In [3]:
total_downdog = len(os.listdir(downdog_dir))
total_goddes = len(os.listdir(goddes_dir))
total_plank = len(os.listdir(plank_dir))
total_tree = len(os.listdir(tree_dir))
total_warrior2 = len(os.listdir(warrior2_dir))

print('file downdog : ',total_downdog)
print('file goddess : ',total_goddes)
print('file plank : ',total_plank)
print('file tree : ',total_tree)
print('file warrior : ',total_warrior2)


file downdog :  97
file goddess :  80
file plank :  115
file tree :  69
file warrior :  109


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode = 'nearest',
    validation_split = 0.2 
)
 
val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode = 'nearest',
    validation_split = 0.2                                  
)

In [8]:
##
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size = (150,150),
    batch_size = 16,
    shuffle = True,
    color_mode = "rgb",
    class_mode = "categorical",
    subset = "training"
    
)

validation_generator = val_datagen.flow_from_directory(
    base_dir,
    target_size = (150,150),
    batch_size = 16,
    shuffle = True,
    color_mode = "rgb",
    class_mode = "categorical",
    subset = "validation"
    
)

Found 378 images belonging to 5 classes.
Found 92 images belonging to 5 classes.


In [9]:
#membuat model class adalah untuk mengukur 

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), strides = (1,1),  activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2), padding = 'valid'), 
    tf.keras.layers.Conv2D(64, (3,3), strides = (1,1), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2), padding = 'valid'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.001))
])


model.summary()

2022-12-04 10:36:09.172818: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 10:36:09.173526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 10:36:09.173726: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-04 10:36:09.173865: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-04 10:36:09.173993: W tensorflow/c

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dropout (Dropout)           (None, 82944)             0

In [27]:
Adam(learning_rate=0.001, name='Adam')
model_vis = model.compile(
    optimizer = 'Adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])


In [28]:
def scheduler(epoch, lr):
  if epoch < 5:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs', histogram_freq=0, write_graph=True, write_images=False,
    update_freq='epoch', embeddings_freq=0,
    embeddings_metadata=None
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dropout (Dropout)           (None, 82944)             0

In [29]:
accuracy = 0.97

class func_callback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy') >= accuracy):
               print('tingkat akurasi : ',accuracy),
               self.model.stop_training = True

callback = func_callback()

In [31]:
batch_size = 16

history = model.fit(
                  train_generator,
                  steps_per_epoch=378//batch_size,
                  epochs=20,
                  validation_data=validation_generator,
                  validation_steps=92//batch_size,
                  verbose=2,
                  callbacks=[lr_schedule, callback]
                  )


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/20
23/23 - 17s - loss: 0.8025 - accuracy: 0.6961 - val_loss: 1.0121 - val_accuracy: 0.6750 - lr: 0.0010 - 17s/epoch - 759ms/step

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/20
23/23 - 16s - loss: 0.7312 - accuracy: 0.7293 - val_loss: 1.0175 - val_accuracy: 0.6625 - lr: 0.0010 - 16s/epoch - 685ms/step

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/20
23/23 - 16s - loss: 0.6577 - accuracy: 0.7845 - val_loss: 0.8423 - val_accuracy: 0.7375 - lr: 0.0010 - 16s/epoch - 700ms/step

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/20
23/23 - 15s - loss: 0.6549 - accuracy: 0.7680 - val_loss: 0.9523 - val_accuracy: 0.6750 - lr: 0.0010 - 15s/epoch - 671ms/step

Epoch 5: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 5/20
23/23 - 19s - loss: 0.6070 - accuracy: 0.

In [2]:
#Mengambil Nilai Accuracy
# visualisasi hasil data training (Plot accuracy & loss model)

import numpy as np
import keras.utils as image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
 

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(20)

plt.figure(figsize=(7, 3))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: name 'history' is not defined

In [36]:
MODEL_BASE_PATH = "model"
SAVE_MODEL_NAME = "model.h5"
save_model_path = os.path.join(MODEL_BASE_PATH,SAVE_MODEL_NAME)
if os.path.exists(os.path.join(MODEL_BASE_PATH)) == False:
    os.makedirs(os.path.join(MODEL_BASE_PATH))
    
print('Saving Model At {}...'.format(save_model_path))
model.save(save_model_path,include_optimizer=False)

Saving Model At model/model.h5...


In [38]:
import fileinput

uploaded = fileinput.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
 
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
 
  classes = model.predict(images, batch_size=16)  

  print(fn)
  if classes[0, 0] == 1:
    print('downdog')
  elif classes[0, 1] == 1:
    print('goddes')
  elif classes[0, 2] == 1:
    print('plank')
  elif classes[0, 3] == 1:
    print('tree')
  elif classes[0, 4] == 1:
    print('warrior2')

AttributeError: module 'fileinput' has no attribute 'upload'

In [39]:
from tkinter import *




gui = Tk(className='Python Examples - Window Color')

gui.geometry("500x500")

gui['bg']='dimgray'

BtnStart = Button(text ="Start", fg='black')
BtnStart.place(x=50, y = 100)

    
BtnStart = Button(text ="Stop", command=vidioStop)
BtnStart.place(x=350, y = 101)

BtnStart = Button(text ="Stop",bg='red')
BtnStart.place(x=200, y = 200)





gui.mainloop()